## Homogeneous collections

The programmer's life would be very boring if you could only work with single entities. In the vast majority of cases, you have collections of data that you have to code with. Although it is possible to build data aggregates using the algebraic types that F# has, the language provides different options to handle collections of data. F# does not have one, but several data types to handle collections, which we will see in the next classes.

In F#, collections are always homogeneous, that is, they contain the same data type. On the other hand, the language provides _immutable_ collection types (`list`, `seq`, `map`, `set`), except `Array` which can be mutable.

The language provides a variety of methods for working with collections as a whole, without the need to iterate to take advantage of them.

### Types and recursion

We have seen how recursion allows us to replace the traditional `for` loop. It is also possible to define recursive types.

For example, one can model the natural numbers by [peano style](https://en.wikipedia.org/wiki/Peano_axioms):

In [52]:
type Peano = 
    | Zero 
    | Succ of Peano

In [53]:
let one = Succ Zero 
let two = Succ(Succ(Zero))

printfn "%A" one
printfn "%A" two


Succ Zero
Succ (Succ Zero)


Like recursion in algorithms, recursive types always have one or more _base_ cases, and then one or more recursive cases.

With this possibility, we can construct a recursive data type that corresponds to a linked list (_linked list_), where each node of the list contains an element and the rest of the list:

In [1]:
type MyList =
    | Empty
    | Cons of int * MyList 

In [2]:
let emptyList  = Empty

let singletonList = Cons (3, Empty)

let exampleList = Cons (1, Cons (2, Cons (3, Empty)))

printfn "%A" exampleList

Cons (1, Cons (2, Cons (3, Empty)))


Of course one would need some functions to populate this list. For example, `replicate x n` creates a list of length `n` with the same element `x`:

In [3]:
let rec replicate x n =
    if n = 0 then
        Empty 
    else
        Cons(x, replicate x (n - 1))

In [4]:
replicate 1 4

Cons (1, Cons (1, Cons (1, Cons (1, Empty)))) Item1 1 Item2 Cons (1, Cons (1, Cons (1, Empty))) Item1 1 Item2 Cons (1, Cons (1, Empty)) Item1 1 Item2 Cons (1, Empty) Item1 1 Item2 Empty

Or you could also create a list of n elements from 0 to n-1:

In [5]:
let initList n =

    let rec loop i n = 
        if n = 0 then
            Empty
        else
            if (i<n) then 
                Cons(i, loop (i+1) n )
            else 
                Cons(n,Empty)


    loop 0 (n-1) 

In [6]:
printfn "%A" (initList 2)
printfn "%A" (initList 3)
printfn "%A" (initList 4)

Cons (0, Cons (1, Empty))
Cons (0, Cons (1, Cons (2, Empty)))
Cons (0, Cons (1, Cons (2, Cons (3, Empty))))


Thus it is possible then to create collections of elements of the same type recursively.

## Operations on lists

In essence, there are a few operations that can be performed on collections of entities of the same type. For example, one could apply a function to each item in the list, to make it another item. Or a value could be obtained from a computation over all the elements of a list (calculate the maximum or the minimum, obtain the
average, etc.). When using a functional style, it is important to focus the programming of
code that uses collections with these operations in mind. In the case of F#, there is a great variety of methods on the different types of collections that the language provides. In most cases it is possible to combine these methods to get the required result, without having to use recursion.

Continuing with the `MyList` example, we are going to define two types of operations on collections that are very important.

The first case is that of `map`, which allows you to apply a function to each element of the list:

In [7]:
let rec map f list =
    match list with 
    | Empty -> Empty 
    | Cons (n, ls) -> 
        Cons(f n, map f ls)

In [8]:
let square i = 
    i*i 

let l = initList 4

let lsq = map square l 

printfn "%A" l
printfn "%A" lsq

Cons (0, Cons (1, Cons (2, Cons (3, Empty))))
Cons (0, Cons (1, Cons (4, Cons (9, Empty))))


The following is an example of the reduction operation, which consists of applying a function that combines two elements of the list successively:

In [9]:
let fold f list =

    let rec loop acc f list = 
        match list with 
        | Empty -> acc 
        | Cons (n, ls) -> 
            let a = f acc n 
            loop a f ls 

    loop 0 f list             

In [10]:
let add i j : int = 
    i + j 

let l2 = initList 5
let sum  = fold add l2 

printfn "%A" sum 

10


These methods can be properly composed either through _piping_:

In [64]:
l2
|> map square
|> fold add

30

Or by defining a composition of functions:

In [65]:
let squareAndAdd = (map square) >> (fold add) 

printfn "%A" (squareAndAdd l2)

30


### Lists

F# provides the `list` type to handle lists as we saw in the previous example, with a clear syntax: square brackets are used to define the beginning and end of the list, which contains elements separated by `;`.

In [13]:
let l3 = [0; 2; 3]

let singleton = ["Hola"]

let empty = List.empty 

printfn "%A" l3
printfn "%A" empty

printfn "%A" l3[1]
printfn "%A" l3.[3]


[0; 2; 3]
[]
2


Error: System.ArgumentException: The index was outside the range of elements in the list. (Parameter 'n')
   at Microsoft.FSharp.Collections.PrivateListHelpers.nth[a](FSharpList`1 l, Int32 n) in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4024
   at Microsoft.FSharp.Collections.FSharpList`1.get_Item(Int32 index) in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4085
   at <StartupCode$FSI_0056>.$FSI_0056.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

Lists can be initialized:

In [15]:
// A range list
let range = [0..4]

// A range list with steps 
let rangeby3 = [0..3..17]

// All the same
let constants = List.replicate 4 1

// A range 
let rangeByInit = List.init 4 id 

printfn "%A" range
printfn "%A" rangeby3
printfn "%A" constants
printfn "%A" rangeByInit


[0; 1; 2; 3; 4]
[0; 3; 6; 9; 12; 15]
[1; 1; 1; 1]
[0; 1; 2; 3]


And it provides the corresponding `map` and `fold` functions.

In [16]:
let l4 = List.init 5 id 

let l5 = List.map square l4 

let sum  = List.fold (fun a elem -> a + elem) 0 l4  

printfn "%A" l5 
printfn "%A" sum 

[0; 1; 4; 9; 16]
10


But much more, because the `sum` function does what one supposes:

In [70]:
l4
|> List.map square
|> List.sum 

List.collect 

30

I recommend the page [Choosing between collection functions](https://fsharpforfunandprofit.com/posts/list-module-functions/) to query examples and usage of collections methods.